In [ ]:
%%capture
# install the latest version of kaggle_environments
!pip install --upgrade kaggle_environments

I have borrowed both the agents from : Kore Intro IV: Combat

Notebook Link : https://www.kaggle.com/code/bovard/kore-intro-iv-combat

In [ ]:
%%writefile starter.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if shipyard.ship_count > 10:
            direction = Direction.from_index(turn % 4)
            action = ShipyardAction.launch_fleet_with_flight_plan(randint(2,3), direction.to_char())
            shipyard.next_action = action
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            shipyard.next_action = action
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
            kore_left -= spawn_cost

    return me.next_actions

In [ ]:
%%writefile attacker.py
# from https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/kore_fleets/starter_bots/python/main.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def get_closest_enemy_shipyard(board, position, me):
    min_dist = 1000000
    enemy_shipyard = None
    for shipyard in board.shipyards.values():
        if shipyard.player_id == me.id:
            continue
        dist = position.distance_to(shipyard.position, board.configuration.size)
        if dist < min_dist:
            min_dist = dist
            enemy_shipyard = shipyard
    return enemy_shipyard


def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        action = None
        if turn % 100 < 20 and shipyard.ship_count >= 50:
            closest_enemy_shipyard = get_closest_enemy_shipyard(board, shipyard.position, me)
            if not closest_enemy_shipyard:
                continue
            enemy_pos = closest_enemy_shipyard.position
            my_pos = shipyard.position
            flight_plan = "N" if enemy_pos.y > my_pos.y else "S"
            flight_plan += str(abs(enemy_pos.y - my_pos.y) - 1)
            flight_plan += "W" if enemy_pos.x < my_pos.x else "E"
            action = ShipyardAction.launch_fleet_with_flight_plan(shipyard.ship_count, flight_plan)            
        elif shipyard.ship_count >= 10 and turn % 7 == 0 and turn % 100 > 20 and turn % 100 < 90:
            direction = Direction.from_index(turn % 4)
            opposite = direction.opposite()
            flight_plan = direction.to_char() + "9" + opposite.to_char()
            action = ShipyardAction.launch_fleet_with_flight_plan(10, flight_plan)
        elif kore_left > spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left > spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            kore_left -= spawn_cost
        shipyard.next_action = action

    return me.next_actions

In [ ]:
!mkdir snapshots
starter_agent_path = "/kaggle/working/starter.py"
attacker_agent_path = "/kaggle/working/attacker.py"
from kaggle_environments import make
env = make("kore_fleets", debug=True, configuration={"annotations": True, "width":12, "height":12})
steps = env.run([attacker_agent_path, attacker_agent_path, attacker_agent_path, starter_agent_path])

In [ ]:
env.render(mode="ipython", width=1000, height=800)

In [ ]:
kore = []
for tu in range(len(steps)):
    kore_snap = []
    for i in range(1,22):
        kore_snap.append(steps[tu][0]['observation']['kore'][21*(i-1):21*i])
    kore.append(kore_snap)

## Kore Plot w.r.t Turn

### Initially...

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (40,20)
plt.imshow(kore[0], interpolation='nearest',cmap=plt.cm.get_cmap('Reds', 6))
#plt.rcParams["figure.figsize"] = (40,20)
cbar= plt.colorbar()
cbar.set_label("Kore Count")
plt.show()

### At halftime

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (40,20)
plt.imshow(kore[len(steps)//2], interpolation='nearest',cmap=plt.cm.get_cmap('Reds', 6))
cbar= plt.colorbar()
cbar.set_label("Kore Count")
plt.show()

### End of Game

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (40,20)
plt.imshow(kore[-1], interpolation='nearest',cmap=plt.cm.get_cmap('Reds', 6))
cbar= plt.colorbar()
cbar.set_label("Kore Count")
plt.show()

### At any specific step..

In [ ]:
Turn = 120 # turn number ..vary this to get different results

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (40,20)
plt.imshow(kore[Turn-1], interpolation='nearest',cmap=plt.cm.get_cmap('Reds', 6))
cbar= plt.colorbar()
cbar.set_label("Kore Count")
plt.show()

## Explanation Needed..
Can someone explain to me what this next code segment signify. 

In [ ]:
steps[Turn][0]['observation']['players']

**For turn 0 it is as follows..**

In [ ]:
steps[0][0]['observation']['players']